In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb


In [25]:
drivers_data_df = pd.read_csv("data/Crash_Reporting_-_Drivers_Data.csv")
incidents_data_df = pd.read_csv("data/Crash_Reporting_-_Incidents_Data_20241111.csv")
nonmotorist_data_df = pd.read_csv("data/Crash_Reporting_-_Non-Motorists_Data_20241111.csv")

/tmp/ipykernel_6632/2706314688.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  drivers_data_df = pd.read_csv("data/Crash_Reporting_-_Drivers_Data.csv")
/tmp/ipykernel_6632/2706314688.py:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents_data_df = pd.read_csv("data/Crash_Reporting_-_Incidents_Data_20241111.csv")


In [26]:
drivers_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187311 entries, 0 to 187310
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Report Number                  187311 non-null  object 
 1   Local Case Number              187311 non-null  object 
 2   Agency Name                    187311 non-null  object 
 3   ACRS Report Type               187311 non-null  object 
 4   Crash Date/Time                187311 non-null  object 
 5   Route Type                     168981 non-null  object 
 6   Road Name                      168141 non-null  object 
 7   Cross-Street Name              161910 non-null  object 
 8   Off-Road Description           17292 non-null   object 
 9   Municipality                   19126 non-null   object 
 10  Related Non-Motorist           6009 non-null    object 
 11  Collision Type                 186726 non-null  object 
 12  Weather                       

In [38]:
# incidents_data_df.info()

In [39]:
# nonmotorist_data_df.info()

## Preprocessing

In [67]:
drivers_data_df["Collision Type"] = drivers_data_df["Collision Type"].str.upper()
drivers_data_df["Driver Distracted By"] = drivers_data_df["Driver Distracted By"].str.upper()
drivers_data_df["Vehicle Body Type"] = drivers_data_df["Vehicle Body Type"].str.upper()
drivers_data_df["Injury Severity"] = drivers_data_df["Injury Severity"].str.upper()

drivers_data_df["Driver Substance Abuse"] = drivers_data_df["Driver Substance Abuse"].str.upper()
drivers_data_df["Driver Substance Abuse"] = drivers_data_df["Driver Substance Abuse"].apply(
    lambda desc: list(map(str.strip, desc.split(','))) if isinstance(desc, str) else desc
)


drivers_data_df["Parked Vehicle"] = drivers_data_df["Parked Vehicle"].replace({"Yes" : True,
                                                                              "No" : False})




#Grouping of data
vehicle_body_type_mapping = {
    "Passenger Vehicles": [
        "PASSENGER CAR",
        "(SPORT) UTILITY VEHICLE",
        "SPORT UTILITY VEHICLE",
        "STATION WAGON",
        "AUTOCYCLE",
        "LOW SPEED VEHICLE",
        "RECREATIONAL VEHICLE"
    ],
    "Light Trucks / Vans": [
        "PICKUP TRUCK",
        "PICKUP",
        "OTHER LIGHT TRUCKS (10,000LBS (4,536KG) OR LESS)",
        "VAN",
        "VAN - CARGO",
        "CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))",
        "VAN - PASSENGER (&LT;9 SEATS)",
        "VAN - PASSENGER (9 OR 12 SEATS)"
    ],
    "Heavy/Commercial Vehicles": [
        "MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4,536KG))",
        "TRUCK TRACTOR",
        "SINGLE-UNIT TRUCK",
        "OTHER TRUCKS",
        "CONSTRUCTION EQUIPMENT (BACKHOE, BULLDOZER, ETC.)",
        "FARM VEHICLE",
        "FARM EQUIPMENT (TRACTOR, COMBINE HARVESTER, ETC.)"
    ],
    "Emergency / Law Enforcement Vehicles": [
        "POLICE VEHICLE/NON EMERGENCY",
        "POLICE VEHICLE/EMERGENCY",
        "AMBULANCE/EMERGENCY",
        "AMBULANCE/NON EMERGENCY",
        "FIRE VEHICLE/EMERGENCY",
        "FIRE VEHICLE/NON EMERGENCY"
    ],
    "Buses": [
        "TRANSIT BUS",
        "SCHOOL BUS",
        "BUS - TRANSIT",
        "BUS - SCHOOL",
        "BUS - MINI",
        "BUS - CROSS COUNTRY",
        "CROSS COUNTRY BUS",
        "BUS - OTHER TYPE",
        "OTHER BUS"
    ],
    "Motorcycles / Recreational / Specialty Vehicles": [
        "MOTORCYCLE",
        "MOTORCYCLE - 2 WHEELED",
        "MOTORCYCLE - 3 WHEELED",
        "MOPED",
        "MOPED OR MOTORIZED BICYCLE",
        "ALL TERRAIN VEHICLE (ATV)",
        "ALL-TERRAIN VEHICLE/ALL-TERRAIN CYCLE (ATV/ATC)",
        "SNOWMOBILE",
        "RECREATIONAL OFF-HIGHWAY VEHICLES (ROV)"
    ]
}

vehicle_group_mapping = {vehicle: group for group, vehicles in vehicle_body_type_mapping.items() for vehicle in vehicles}

drivers_data_df["Vehicle Type Group"] = drivers_data_df["Vehicle Body Type"].map(vehicle_group_mapping)


In [68]:
drivers_data_df["Driver Substance Abuse"].value_counts()

Driver Substance Abuse
[NONE DETECTED]                                          122547
[NOT SUSPECT OF ALCOHOL USE, NOT SUSPECT OF DRUG USE]     13073
[UNKNOWN]                                                 11990
[ALCOHOL PRESENT]                                          4087
[UNKNOWN, UNKNOWN]                                         1578
[ALCOHOL CONTRIBUTED]                                      1435
[SUSPECT OF ALCOHOL USE, NOT SUSPECT OF DRUG USE]           389
[ILLEGAL DRUG PRESENT]                                      259
[MEDICATION PRESENT]                                        117
[ILLEGAL DRUG CONTRIBUTED]                                  102
[COMBINED SUBSTANCE PRESENT]                                 92
[MEDICATION CONTRIBUTED]                                     64
[OTHER]                                                      59
[SUSPECT OF ALCOHOL USE, UNKNOWN]                            48
[COMBINATION CONTRIBUTED]                                    47
[UNKNOWN, NOT SUS

In [36]:
drivers_data_df["Light"].value_counts()

Light
DAYLIGHT                    116150
DARK LIGHTS ON               39553
Daylight                     10886
DARK NO LIGHTS                4967
DUSK                          3935
DAWN                          3472
Dark - Lighted                3213
DARK -- UNKNOWN LIGHTING      1578
UNKNOWN                        677
Dark - Not Lighted             522
OTHER                          342
Dusk                           192
Dawn                           164
Dark - Unknown Lighting         97
Other                           64
Unknown                         54
Name: count, dtype: int64

## Load Data to DuckDB